In [99]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
urls = {'node1': 'http://192.168.226.96:8545',
        'node2': 'http://192.168.226.157:8545',
        'node3': 'http://192.168.226.212:8545',
        'node4': 'http://192.168.226.129:8545',
        'node5': 'http://192.168.226.158:8545',
        'node6': 'http://192.168.226.45:8545'
    } 
provider = Web3.HTTPProvider(urls['node1'])
w3 = Web3(provider)
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
# w3.isConnected()
len(w3.geth.admin.peers())

26

In [9]:
for k,v in urls.items():
    provider = Web3.HTTPProvider(v)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    # print(w3.isConnected())
    peers = w3.geth.admin.peers()
    print(k, 'has', len(peers), 'peers')

node1 has 5 peers
node2 has 5 peers
node3 has 5 peers
node4 has 5 peers
node5 has 6 peers
node6 has 5 peers


In [99]:
address1 = '0xfe3b557e8fb62b89f4916b721be55ceb828dbd73'
address2 = '0x627306090abaB3A6e1400e9345bC60c78a8BEf57'
address3 = '0xf17f52151EbEF6C7334FAD080c5704D77216b732'
def get_balance(address):
    balance_bank = w3.eth.getBalance(Web3.toChecksumAddress(address))
    return w3.fromWei(balance_bank, "ether")

print('address1: ', get_balance(address1))
print('address2: ', get_balance(address2))
print('address3: ', get_balance(address3))


address1:  90000
address2:  89995.998488
address3:  90004


In [87]:
# send transactions
buyer_dealer_txn = {
    'from': Web3.toChecksumAddress(address2),
    'to': Web3.toChecksumAddress(address3),
    'value': w3.toWei(1, 'ether'),
    'gas': 90000,
    'gasPrice': 18000000000,
    'nonce': w3.eth.getTransactionCount(address2),
    'chainId': 1337
}

buyer_priv_key = 'c87509a1c067bbde78beb793e6fa76530b6382a4c0241e5e4a9ec0a0f44dc0d3'
signed_txn = w3.eth.account.sign_transaction(buyer_dealer_txn, buyer_priv_key)
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
receipt

AttributeDict({'blockHash': HexBytes('0x2d48e85cee9557f91541415bfc596b2da457cd34f92d09ad800c1b319baca824'),
 'blockNumber': 33639,
 'contractAddress': None,
 'cumulativeGasUsed': 21000,
 'from': '0x627306090abaB3A6e1400e9345bC60c78a8BEf57',
 'gasUsed': 21000,
 'effectiveGasPrice': 18000000000,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': '0xf17f52151EbEF6C7334FAD080c5704D77216b732',
 'transactionHash': HexBytes('0xe2c28b3e055e72ced8521651a5533311f7dd6829c558fc94502

In [88]:
with open('../build/contracts/SimpleStorage.json') as file:
    compiled = json.load(file)
contractAbi = compiled['abi']

contractAddress = '0xF322C60a93c7D8da76722dF93635036ad62f66ff'
# Create the contract instance with the newly-deployed address
contract = w3.eth.contract(
    address=contractAddress,
    abi=contractAbi,
)

print('contract address: ', contract.address)
# Display the default greeting from the contract
print('Default contract simplestorage: {}'.format(
    contract.functions.get().call()
))

# tx_hash = contract.functions.set(120).transact()

# # # Wait for transaction to be mined...
# w3.eth.waitForTransactionReceipt(tx_hash)

# # Display the new storedData value
# print('Updated contract storedData: {}'.format(
#     contract.functions.get().call()
# ))

contract address:  0x7269435b49dFb67Dbe5460fC30Fed2b1d0C01b69
Default contract simplestorage: 100


In [105]:
contracts = ['EnergyCoin', 'PaymentSettlement', 'PoolMarket']
contractAddresses = {'EnergyCoin': '0x8eB9147a7b1cfD5d20dC5bc15ca60c16F0D871cb',
    'PaymentSettlement': '0xf1D795F56996eC57820449Ee14Dd710dAb0c8E8f',
    'PoolMarket': '0x122Aab2C3A3f73200315b2e417236259415c7C25'}
contracts = contractAddresses.keys()

abis = {}
for contract in list(contracts):
    with open('../build/contracts/{}.json'.format(contract)) as file:
        compiled = json.load(file)
    contractAbi = compiled['abi']
    abis[contract] = contractAbi

In [111]:
EnergyCoin = w3.eth.contract(
    address=contractAddresses['EnergyCoin'],
    abi=abis['EnergyCoin'],
)

EnergyCoin.functions.getMyOwnBalance().call()

0

In [114]:
priKey = '33ed0291ce66f3f6f5e14a45a6edfaf375a9a8b8591f968ce8c42fb8f01498f3'
Web3.toChecksumAddress(address1)

'0xFE3B557E8Fb62b89F4916B721be55cEb828dBd73'

In [2]:
import pandas as pd
import numpy as np
import ast
import redis
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware

# connect Redis databases
WATCHDOG_ADDRESS ="192.168.226.163"
db1 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=1)
db1_keys = db1.keys()

db2 = redis.StrictRedis(
    host=WATCHDOG_ADDRESS,
    port=6379,
    db=2)
db2_keys = db2.keys()
genesis = ast.literal_eval(db2.get(b'genesis').decode('utf-8'))

# construct dataframe
rows = []
for key in db1_keys:
    row = []
    # IP address of node
    ip = key.decode("utf-8")
    # check if node is validator
    value = ast.literal_eval(db1.get(key).decode('utf-8'))
    is_validator = value['key'][2:] in genesis['extraData']
    # find the number of peers
    url = 'http://'+ip+':8545'
    provider = Web3.HTTPProvider(url)
    w3 = Web3(provider)
    w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    peers_num = len(w3.geth.admin.peers())

    row.append(value['hostname'])
    row.append(ip)
    row.append(value['key'])
    row.append(is_validator)
    row.append(peers_num)
    
    rows.append(row)
df_orig = pd.DataFrame(np.array(rows),
    columns=['Hostname', 'IP', 'NodeAddress', 'IsValidator', 'PeersNum'])

# sort dataframe
df = df_orig.copy()
idx = []
for name in df.Hostname:
    idx.append(int(name.split('-')[1]))
df['Index'] = idx
df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
df

,Hostname,IP,NodeAddress,IsValidator,PeersNum
Index,,,,,
1,besu-1,192.168.226.166,0x4a0f9df01cc7de67055963153d76788c0cafee6c,True,25
2,besu-2,192.168.226.44,0x36e2ad0716c214ee232fb1c1b3361d73688f0fda,True,26
3,besu-3,192.168.226.189,0xdb8a1299906804d887a6c0a66b417884046b6e91,True,25
4,besu-4,192.168.226.96,0xbf23e801b516ad1530fb7cbcd3b21dd894975e99,True,26
5,besu-5,192.168.226.111,0x18f1e29e48022f5695b1fbb3dd9d1148c96e0ed1,True,119
6,besu-6,192.168.226.129,0xacabe4aedddb14cb9e7bb89677406dac386fc32f,True,69
7,besu-7,192.168.226.165,0xa4099866fab80c2d2aa18417e554fe5599fa9661,True,26
8,besu-8,192.168.226.208,0x1ea9ed79bc47381c21cc6d2a083b5840375c268e,True,25
9,besu-9,192.168.226.56,0xc5358ab4635a1b8b7c516064bf3166511ece261d,True,25


In [3]:
df.to_csv('network_data.csv')